In [7]:
from scrapping_sources.Nasdaq import Nasdaq as N
import datetime

df = N().earnings_release('2021-08-27')
print(df)
# try:
res =  df['symbol'].values
res
# except:
#     res =  df['ticker'].values
# finally:
#     res = None
# N().earnings_release_tickers(datetime.date.today() -datetime.timedelta(days=11))
# N().earnings_release_tickers('2021-08-27')

        eps surprise               time symbol  \
0     $4.90      N/A  time-not-supplied    PKX   
1     $2.46     4.68  time-not-supplied     UI   
2   ($2.91)     4.28  time-not-supplied     BZ   
3     $0.64      N/A  time-not-supplied     KT   
4     $0.11      N/A  time-not-supplied    SHI   
5     $1.09    -3.54  time-not-supplied    BIG   
6     $2.86   130.65  time-not-supplied   HIBB   
7      $999      N/A  time-not-supplied    MTL   
8   ($0.30)      N/A  time-not-supplied     YI   
9   ($0.05)      N/A  time-not-supplied   MOGU   
10    $0.08      N/A  time-not-supplied   KFFB   
11  ($0.12)      -20  time-not-supplied   BMRA   
12    $0.02      N/A  time-not-supplied   TAYD   

                                           name        marketCap  \
0                                         POSCO  $22,221,818,076   
1                                 Ubiquiti Inc.  $19,096,674,559   
2                               KANZHUN LIMITED  $15,541,195,198   
3                          

In [13]:
import requests
import time

time.sleep(1)
# print(10)
file = f'https://finviz.com/screener.ashx?v=152&c=1,2,3,4,5&r={str(10)}'
headers = {
'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'
}
res = requests.get(file, headers=headers)
# data = pd.read_html(res.content)[7]
print(res.status_code)


200


In [6]:
M.arrange_data('CANF', 'cash-flow-statement', "quarterly")

CANF <Response [200]>


,date,statement,ticker,security_id,line_item,amount
0,2021-06-30,CF-Q,CANF,None,Total Depreciation And Amortization - Cash Flow,0.008
1,2021-06-30,CF-Q,CANF,None,Other Non-Cash Items,0.038
2,2021-06-30,CF-Q,CANF,None,Total Non-Cash Items,0.046
3,2021-06-30,CF-Q,CANF,None,Change In Accounts Receivable,-0.688
4,2021-06-30,CF-Q,CANF,None,Change In Inventories,0.0
...,...,...,...,...,...,...
695,2011-12-31,CF-Q,CANF,None,Financial Activities - Other,3.63
696,2011-12-31,CF-Q,CANF,None,Cash Flow From Financial Activities,4.95
697,2011-12-31,CF-Q,CANF,None,Net Cash Flow,-0.8
698,2011-12-31,CF-Q,CANF,None,Stock-Based Compensation,0.0
